### Part 1. Web Scraping ###

In [27]:
# import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import time
import pandas

from webdriver_manager.chrome import ChromeDriverManager

/Users/kevinlee/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [4]:
driver_path=ChromeDriverManager().install()
print(f'driver_path is a {type(driver_path)} with a value of {driver_path}')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 110.0.5481
[WDM] - Get LATEST driver version for 110.0.5481


[WDM] - Driver [/Users/kevinlee/.wdm/drivers/chromedriver/mac64/110.0.5481.77/chromedriver] found in cache


driver_path is a <class 'str'> with a value of /Users/kevinlee/.wdm/drivers/chromedriver/mac64/110.0.5481.77/chromedriver


In [5]:
# create browser
browser=Browser('chrome', executable_path=driver_path)

In [6]:
URL='https://www.rottentomatoes.com/browse/movies_in_theaters/'

# visit url
browser.visit(URL)

In [7]:
# get html from browser
html=browser.html
print(f'html is a {type(html)} with a value of {html}')

html is a <class 'str'> with a value of <html lang="en" dir="ltr" xmlns="http://www.w3.org/1999/xhtml" prefix="fb: http://www.facebook.com/2008/fbml og: http://opengraphprotocol.org/schema/"><head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#" class="at-element-marker"><script type="text/javascript" async="" src="https://static.criteo.net/js/ld/publishertag.prebid.132.js"></script><script async="" src="//tags.bkrtx.com/js/bk-coretag.js"></script><script async="" src="https://cdn.jsdelivr.net/npm/search-insights@1.5.0"></script><script async="" src="https://honorableland.com/v2xglf05UGcVvzEUFlB6672D6fDwhuTsalYNh9J8Q6zzvZToUzEeznRfzQK-ELINP"></script><script type="text/javascript" async="" src="//static.chartbeat.com/js/chartbeat.js"></script><script async="" src="https://sb.scorecardresearch.com/beacon.js"></script><script type="text/javascript" async="" src="https://cdn.quantummetric.com/qscripts/quantum-rottentomatoes.js"></script><script asyn

In [10]:
# create soup object with html
soup=BeautifulSoup(html, 'html.parser')

#### optionally save HTML file ####

In [12]:
with open('data/original.html', 'w') as f: 
    f.write(html)

In [15]:
# parse element from html
# find the tiles division
tiles=soup.find_all('div', class_='discovery-tiles')[1]

links_list=tiles.find_all('a', class_='js-tile-link')

In [21]:
movie_dict={}

current_movie=links_list[0]
title=current_movie.find_all('span', class_='p--small')[0].text.strip()
open_date=current_movie.find_all('span', class_='smaller')[0].text.strip()
movie_link=current_movie['href']
poster_link=current_movie.find_all('img', class_='posterImage')[0]['src']
score_pairs=current_movie.find_all('score-pairs')[0]

# get attributes from score-pairs
audience_score=None
audience_sentiment=None
critics_score=None
critics_sentiment=None

try: 
    audience_score=score_pairs['audiencescore']
except: 
    pass

try: 
    audience_sentiment=score_pairs['audience_sentiment']
except: 
    pass

try: 
    critics_score=score_pairs['critics_score']
except: 
    pass

try: 
    critics_sentiment=score_pairs['critics_sentiment']
except: 
    pass

# store data as dictionary
movie_dict['title']=title
movie_dict['open_date']=open_date
movie_dict['movie_link']=movie_link
movie_dict['poster_link']=poster_link

movie_dict

{'title': 'The Thorn',
 'open_date': 'Opened Mar 06, 2023',
 'movie_link': '/m/the_thorn_2023',
 'poster_link': 'https://resizing.flixster.com/bFeiUQwfc2drDpWul3gV3g9PAXk=/180x258/v2/https://resizing.flixster.com/m9SUebSjlMIrn5G9VEGmx6mKL3c=/ems.cHJkLWVtcy1hc3NldHMvbW92aWVzL2ZmNmVhOThmLWZiN2EtNGMzOS1hODc5LThjNTI5YjU4OTc4Yy5qcGc='}

In [26]:
# set JS script to scroll
SCROLL_JS='window.scrollTo(0, document.body.scrollHeight)'

# tell browser to execute JS
browser.execute_script(SCROLL_JS)

# click button
more_button=browser.find_by_text('Load more')
more_button.click()

### Putting it Together ###

In [30]:
URL='https://www.rottentomatoes.com/browse/movies_in_theaters/'

# visit url
browser.visit(URL)

time.sleep(2)

for i in range(3): 
    # set JS script to scroll
    SCROLL_JS='window.scrollTo(0, document.body.scrollHeight)'

    # tell browser to execute JS
    browser.execute_script(SCROLL_JS)

    # click button
    more_button=browser.find_by_text('Load more')
    more_button.click()
    
    time.sleep(2)

# get html from browser
html=browser.html

# create soup object with html
soup=BeautifulSoup(html, 'html.parser')    

# parse element from html
# find the tiles division
tiles=soup.find_all('div', class_='discovery-tiles')[1]

links_list=tiles.find_all('a', class_='js-tile-link')

# create data list to hold all movies
data=[]
for each_movie in links_list: 
    movie_dict={}

    current_movie=each_movie
    title=current_movie.find_all('span', class_='p--small')[0].text.strip()
    open_date=current_movie.find_all('span', class_='smaller')[0].text.strip()
    movie_link=current_movie['href']
    poster_link=current_movie.find_all('img', class_='posterImage')[0]['src']
    score_pairs=current_movie.find_all('score-pairs')[0]

    # get attributes from score-pairs
    audience_score=None
    audience_sentiment=None
    critics_score=None
    critics_sentiment=None

    try: 
        audience_score=score_pairs['audiencescore']
    except: 
        pass

    try: 
        audience_sentiment=score_pairs['audience_sentiment']
    except: 
        pass

    try: 
        critics_score=score_pairs['critics_score']
    except: 
        pass

    try: 
        critics_sentiment=score_pairs['critics_sentiment']
    except: 
        pass

    # store data as dictionary
    movie_dict['title']=title
    movie_dict['open_date']=open_date
    movie_dict['movie_link']=movie_link
    movie_dict['poster_link']=poster_link
    movie_dict['critic_score']=critic_score
    movie_dict['critic_sentiment']=critic_sentiment
    movie_dict['audience_score']=audience_score
    movie_dict['audience_sentiment']=audience_sentiment

    data.append(movie_dict)

In [31]:
data

[{'title': 'The Thorn',
  'open_date': 'Opened Mar 06, 2023',
  'movie_link': '/m/the_thorn_2023',
  'poster_link': 'https://resizing.flixster.com/bFeiUQwfc2drDpWul3gV3g9PAXk=/180x258/v2/https://resizing.flixster.com/m9SUebSjlMIrn5G9VEGmx6mKL3c=/ems.cHJkLWVtcy1hc3NldHMvbW92aWVzL2ZmNmVhOThmLWZiN2EtNGMzOS1hODc5LThjNTI5YjU4OTc4Yy5qcGc='},
 {'title': 'Tu Jhoothi Main Makkaar',
  'open_date': 'Opens Mar 08, 2023',
  'movie_link': '/m/tu_jhoothi_main_makkaar',
  'poster_link': 'https://resizing.flixster.com/fQ4D1iiRPiwNdFXGE6za19k-bhA=/180x258/v2/https://resizing.flixster.com/vqEGJotvD5G3tMkJUb8eBWKC_vk=/ems.cHJkLWVtcy1hc3NldHMvbW92aWVzL2UwZGFhNDY0LWJmZDUtNDRhZC04YzQ2LWEwNmIzN2MzZmQ3Zi5qcGc='},
 {'title': 'Mitran Da Naa Chalda',
  'open_date': 'Opens Mar 08, 2023',
  'movie_link': '/m/mitran_da_naa_chalda',
  'poster_link': 'https://resizing.flixster.com/0q8o60vxY3-QnvPyzmu-p2e1-2U=/180x258/v2/https://resizing.flixster.com/zWOjQYmc6n88sUHCh8DFiIpcA4I=/ems.cHJkLWVtcy1hc3NldHMvbW92aWVzLzBlNDEw

In [33]:
browser.quit()

In [34]:
# import data as DataFrame
df=pandas.DataFrame(data)
df.head()

,title,open_date,movie_link,poster_link
0,The Thorn,"Opened Mar 06, 2023",/m/the_thorn_2023,https://resizing.flixster.com/bFeiUQwfc2drDpWu...
1,Tu Jhoothi Main Makkaar,"Opens Mar 08, 2023",/m/tu_jhoothi_main_makkaar,https://resizing.flixster.com/fQ4D1iiRPiwNdFXG...
2,Mitran Da Naa Chalda,"Opens Mar 08, 2023",/m/mitran_da_naa_chalda,https://resizing.flixster.com/0q8o60vxY3-QnvPy...
3,Uchiyan Ne Gallan Tere Yaar Diyan,"Opens Mar 08, 2023",/m/uchiyan_ne_gallan_tere_yaar_diyan,https://resizing.flixster.com/CHvICSdOHXUD-cfd...
4,Scream VI,"Opens Mar 10, 2023",/m/scream_vi,https://resizing.flixster.com/SNmo199mLTKkD3lO...


In [35]:
# save as CSV
df.to_csv('data/data.csv', index=False)